In [1]:
import pyspark
from pyspark.sql import SparkSession
import os

print("Loading hive-site.xml from", os.environ.get("HADOOP_CONF_DIR"))

conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        .set('spark.sql.catalog.type', 'hive')
        .set('spark.sql.catalog.spark_catalog', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.iceberg_catalog.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
        .set('spark.jars.packages', 'com.amazonaws:aws-java-sdk-bundle:1.11.1026,org.apache.hadoop:hadoop-aws:3.3.2,org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1,org.apache.iceberg:iceberg-spark3-extensions:0.13.1')
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
)

Loading hive-site.xml from /usr/local/spark/conf/


In [2]:
# Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
# Create database
spark.sql("CREATE DATABASE IF NOT EXISTS demo_hms").show()

++
||
++
++



In [4]:
# Create table
spark.sql("CREATE TABLE IF NOT EXISTS demo_hms.currency (name STRING) USING iceberg;").show()

++
||
++
++



In [5]:
# Insert Some Data
spark.sql("INSERT INTO demo_hms.currency VALUES ('EUR'), ('USD'), ('GBP')").show()

++
||
++
++



In [6]:
# Query the Data
spark.sql("SELECT * FROM demo_hms.currency;").show()

+----+
|name|
+----+
| EUR|
| USD|
| GBP|
+----+



In [7]:
# Remove the Data
spark.sql("DELETE FROM demo_hms.currency;").show()

++
||
++
++



In [8]:
# View snapshots history
spark.sql("SELECT * FROM spark_catalog.demo_hms.currency.snapshots;").show()

+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|          parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|2023-08-15 14:06:...| 961821184508105354|               null|   append|s3a://warehouse-h...|{spark.app.id -> ...|
|2023-08-15 14:06:...|2093570691111897546| 961821184508105354|   delete|s3a://warehouse-h...|{spark.app.id -> ...|
|2023-08-15 14:47:...|5686632845824564718|2093570691111897546|   append|s3a://warehouse-h...|{spark.app.id -> ...|
|2023-08-15 14:48:...|6052293513425466728|5686632845824564718|   delete|s3a://warehouse-h...|{spark.app.id -> ...|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+



In [9]:
# View timestamp of snapshot when data was appended(before deletion)
spark.sql("SELECT committed_at FROM spark_catalog.demo_hms.currency.snapshots WHERE operation='append'").show(10, False)

+-----------------------+
|committed_at           |
+-----------------------+
|2023-08-15 14:06:35.964|
|2023-08-15 14:47:56.572|
+-----------------------+



In [10]:
# Let's query the data before deletion with time travel, replace timestamp with commited_at from above
spark.sql("SELECT * FROM spark_catalog.demo_hms.currency TIMESTAMP AS OF '2023-08-15 14:06:35.964'").show()

+----+
|name|
+----+
| EUR|
| USD|
| GBP|
+----+



In [11]:
# Cleanup
spark.sql("DROP TABLE demo_hms.currency PURGE").show()

++
||
++
++

